# markdown.obsidian.personal.vault

> Manage a vault in ways that are specific to `trouver`

In [ ]:
#| default_exp markdown.obsidian.personal.vault

`trouver` currently requires `Obsidian.md` vaults to be setup in specific ways[^1]. 

[^1]: Most representatively, `trouver` curreently requires the vault to have a directories called `_references` and `_templates` in the root directory of the vault. These directories, need to have subdirectories named `A-E`, `F-J`, `K-O`, `P-T`, `U-Z`, each of which need to have further subdirectories whose names are single letter alphabets in the ranges of their own names, e.g. `A-E` has subdirectories named `A`, `B`, `C`, `D`, and `E`. 

    These directories store reference notes and template notes, see ``14_markdown.obsidian.personal.note_type``

In [ ]:
#| export
from pathlib import Path
import os
from os import PathLike

In [ ]:
import shutil
import tempfile
from unittest import mock

from fastcore.test import *
from nbdev.showdoc import show_doc

from trouver.helper import path_name_no_ext, _test_directory

# Setup an `Obsidian.md` vault for using `trouver`

In [ ]:
#| export
def setup_obsidian_vault_for_trouver(
        vault: PathLike, # The path to the vault to setup.
        verbose: bool = True, # If true, print messages explaining what is created
        prompt: bool = True, # If true, prompt user input for certain decisions, such as whether to create an Obsidian vault folder if `vault` does not exist.
    ):
    """Setup an `Obsidian.md` vault for `trouver`.
    
    Currently, the following subdirectories are created in `vault` if they do not
    already exist:

    - `_references`, `_templates`
        - Each of these directories are then populated by subdirectories 
          `A-E`, `F-J`, `K-O`, `P-T`, `U-Z`, and those subdirectories are
          further populated by subdirectories whose names are single-letter
          alphabets in the ranges of these names, e.g. `A-E` has subdirectories
          `A`, `B`, `C`, `D`, and `E`.
    """
    _setup_trouver_vault_essentials(Path(vault), verbose, prompt)


def _setup_trouver_vault_essentials(
        vault: PathLike, # The path to the vault to setup
        verbose: bool, # If true, print messages explaining what is created
        prompt: bool, # If true, prompt user input for certain decisions, such as whether to create an Obsidian vault folder if `vault` does not exist.
    ):
    """
    """
    # TODO: setup missing subfolders in the `_references` or `_templates` subfolders even
    for name in ['_references', '_templates']:
        if os.path.exists(vault / name) and verbose:
            print(f"The vault already has a `{name}` folder.")
        elif not os.path.exists(vault / name) and prompt:
            # TODO do input things
            command = input(
                f"""The vault does not have a `{name}` folder.
                Create it and subdirectories?\n[Y/(n)]""")
        # if not os.path.exists(vault / name) and ()
        #     print('')
        if not os.path.exists(vault / name) and (not prompt or command == 'Y'):
            if verbose:
                print(f"Creating `{name}` folder in the vault.")
            _setup_alphabetical_subdirectories(vault, name)



def _setup_alphabetical_subdirectories(
        dir: PathLike, # The path to the directory in which to make the subdirectory with alphabetical subdirectories
        name: str, # The name of the subdirecctory to make in which the alphabetical subdirectories will be made
    ):
    """Create a subdirectory with name ``name`` in the directory ``dir`` and create
    alphabetical subdirectories.
    
    Assumes that ``dir`` exists, but the subdirectory with name ``name`` does not.
    """
    os.mkdir(dir / name)
    sub_names = ['A-E', 'F-J', 'K-O', 'P-T', 'U-Z']
    for sub_name in sub_names:
        os.mkdir(dir / name / sub_name)
        first_letter, last_letter = ord(sub_name[0]), ord(sub_name[-1])
        for i in range(first_letter, last_letter+1):
            os.mkdir(dir / name / sub_name / chr(i))




In [ ]:
#| hide
with (mock.patch('os.mkdir') as mock_mkdir):
    _setup_alphabetical_subdirectories(Path('.'), '_references')

    assert mock.call(Path('_references') / 'A-E') in mock_mkdir.call_args_list
    assert mock.call(Path('_references') / 'U-Z') in mock_mkdir.call_args_list
    assert mock.call(Path('_references') / 'A-E' / 'A') in mock_mkdir.call_args_list
    assert mock.call(Path('_references') / 'K-O' / 'L') in mock_mkdir.call_args_list

In [ ]:
#| hide
with (mock.patch('__main__.input', return_value='Y') as mock_input,
      mock.patch('__main__._setup_alphabetical_subdirectories') as mock_setup_alphabetical_subdirectories):
    mock_vault = Path('.')
    _setup_trouver_vault_essentials(vault=mock_vault, verbose=True, prompt=True)

    test_eq(
        mock_setup_alphabetical_subdirectories.call_args_list,
        [mock.call(mock_vault, '_references'), mock.call(mock_vault, '_templates')])



Creating `_references` folder in the vault.
Creating `_templates` folder in the vault.


In [ ]:
# TODO: test setup_obsidian_vault_for_trouver

with (tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir):
    # Vault without the '_references' and '_templates' folder.
    # The '_references' and '_templates' folders are created with subdirectories.
    temp_vault = Path(temp_dir) / 'test_vault_1'
    shutil.copytree(_test_directory() / 'test_vault_1', temp_vault)

    setup_obsidian_vault_for_trouver(temp_vault, False, False)
    assert os.path.exists(Path(temp_dir) / 'test_vault_1' / '_references')
    assert os.path.exists(Path(temp_dir) / 'test_vault_1' / '_references' / 'A-E')
    assert os.path.exists(Path(temp_dir) / 'test_vault_1' / '_references' / 'A-E' / 'A')
    assert os.path.exists(Path(temp_dir) / 'test_vault_1' / '_references' / 'A-E' / 'B')
    assert os.path.exists(Path(temp_dir) / 'test_vault_1' / '_references' / 'A-E' / 'C')
    assert os.path.exists(Path(temp_dir) / 'test_vault_1' / '_references' / 'A-E' / 'D')
    assert os.path.exists(Path(temp_dir) / 'test_vault_1' / '_references' / 'A-E' / 'E')
    assert os.path.exists(Path(temp_dir) / 'test_vault_1' / '_references' / 'F-J')
    assert os.path.exists(Path(temp_dir) / 'test_vault_1' / '_references' / 'K-O')
    assert os.path.exists(Path(temp_dir) / 'test_vault_1' / '_references' / 'P-T')
    assert os.path.exists(Path(temp_dir) / 'test_vault_1' / '_references' / 'U-Z')

    assert os.path.exists(Path(temp_dir) / 'test_vault_1' / '_templates')
    assert os.path.exists(Path(temp_dir) / 'test_vault_1' / '_templates' / 'A-E')
    assert os.path.exists(Path(temp_dir) / 'test_vault_1' / '_templates' / 'F-J')
    assert os.path.exists(Path(temp_dir) / 'test_vault_1' / '_templates' / 'K-O')
    assert os.path.exists(Path(temp_dir) / 'test_vault_1' / '_templates' / 'K-O' / 'K')
    assert os.path.exists(Path(temp_dir) / 'test_vault_1' / '_templates' / 'K-O' / 'L')
    assert os.path.exists(Path(temp_dir) / 'test_vault_1' / '_templates' / 'K-O' / 'M')
    assert os.path.exists(Path(temp_dir) / 'test_vault_1' / '_templates' / 'K-O' / 'N')
    assert os.path.exists(Path(temp_dir) / 'test_vault_1' / '_templates' / 'K-O' / 'O')
    assert os.path.exists(Path(temp_dir) / 'test_vault_1' / '_templates' / 'P-T')
    assert os.path.exists(Path(temp_dir) / 'test_vault_1' / '_templates' / 'U-Z')


with (tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir):
    # Vault already with the '_references' and '_templates' folder
    # The `_references` and `_templates` folder do not get re-made.
    temp_vault = Path(temp_dir) / 'empty_model_vault'
    shutil.copytree(_test_directory() / 'empty_model_vault', temp_vault)
    with (mock.patch('os.mkdir') as mock_mkdir):
        assert os.path.exists(Path(temp_dir) / 'empty_model_vault' / '_references')
        setup_obsidian_vault_for_trouver(temp_vault, False, False)
        assert os.path.exists(Path(temp_dir) / 'empty_model_vault' / '_references')
        mock_mkdir.assert_not_called()
